In [17]:
import os

import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer
import torch
from pathlib import Path
import numpy as np

In [2]:
torch.cuda.is_available()

True

In [3]:
MODEL = '../../Sentiment_Analysis/turkish-sentiment-XMLRoBERTa/2_Label_Twitter/'

In [4]:
tokenizer = AutoTokenizer.from_pretrained('cardiffnlp/twitter-xlm-roberta-base', use_fast=True)

In [5]:
data = pd.read_csv(f'../../StockAndTweetDataExtract/Bist100/2022/tweet_part1.csv')
for i in range(2,4):
    data = pd.concat([data, pd.read_csv(f'../../StockAndTweetDataExtract/Bist100/2022/tweet_part{i}.csv')])

In [6]:
data.drop_duplicates(subset=['Tweet'])
data

,Date,User,Tweet
0,2022-10-18 23:55:27+00:00,MikroTrader_,#vestl karışık çizimler ile nadir yapılan bir ...
1,2022-10-18 23:30:25+00:00,akaya006,#ygyo akıl suzgecinizden gecirin #bist100 #BI...
2,2022-10-18 23:26:47+00:00,muratkazan55,@erhantayfur1648 Bizim kitabımızda nakite geçm...
3,2022-10-18 23:16:40+00:00,firattsahin,"18.10.2022 tarihinde 50,85 -53,90 TL fiyat ara..."
4,2022-10-18 23:12:58+00:00,seans_borsa,Borsa İstanbul günü yatay seyirde kapattı\n #B...
...,...,...,...
543,2022-03-18 02:30:22+00:00,ForexNewss,"ABD Başkanı Biden, Çin Devlet Başkanı Xi ile 1..."
544,2022-03-18 02:00:39+00:00,hisseneoldu,⏰ 05:00 \n📊 TUPRS = ₺ \n#tuprs #borsa #bist100
545,2022-03-18 01:38:45+00:00,trend_is_frnd,"#tknsa bu ucgen kirildiktan sonra , tahtaci ne..."
546,2022-03-18 01:21:08+00:00,ForexNewss,"ABD basınında çıkan haberlere göre Putin, isti..."


In [7]:
data_p = data.Tweet.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=256))
data_p

0      [input_ids, attention_mask]
1      [input_ids, attention_mask]
2      [input_ids, attention_mask]
3      [input_ids, attention_mask]
4      [input_ids, attention_mask]
                  ...             
543    [input_ids, attention_mask]
544    [input_ids, attention_mask]
545    [input_ids, attention_mask]
546    [input_ids, attention_mask]
547    [input_ids, attention_mask]
Name: Tweet, Length: 301706, dtype: object

In [8]:
data['Tweet'] = data_p
data_p = 0

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)
classifier = Trainer(model)

In [10]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings['Tweet']

    def __getitem__(self, idx):
        item = {"input_ids": torch.tensor(self.encodings.iloc[idx]["input_ids"])}
        item['attention_mask'] = torch.tensor(self.encodings.iloc[idx]["attention_mask"])
        return item

    def __len__(self):
        return len(self.encodings)

dataset = MyDataset(data)

In [11]:
test_preds_raw, test_labels , _ = classifier.predict(dataset)

***** Running Prediction *****
  Num examples = 301706
  Batch size = 8


In [16]:
test_preds_raw[301705]

array([-1.8322736,  1.9635969], dtype=float32)

In [35]:
final_data = pd.concat([data['Date'].reset_index(drop=True), pd.DataFrame(np.argmax(test_preds_raw, axis=-1), columns=['Sentiment'])], axis=1)
final_data

,Date,Sentiment
0,2022-10-18 23:55:27+00:00,0
1,2022-10-18 23:30:25+00:00,1
2,2022-10-18 23:26:47+00:00,1
3,2022-10-18 23:16:40+00:00,0
4,2022-10-18 23:12:58+00:00,1
...,...,...
301701,2022-03-18 02:30:22+00:00,1
301702,2022-03-18 02:00:39+00:00,0
301703,2022-03-18 01:38:45+00:00,1
301704,2022-03-18 01:21:08+00:00,0


In [ ]:
final_data.to_csv('2022.csv')